In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"
ratings_url="https://datasets.imdbws.com/title.ratings.tsv.gz"
akas_url="https://datasets.imdbws.com/title.akas.tsv.gz"

In [3]:
basics = pd.read_csv(basics_url,sep='\t', low_memory=False)
ratings = pd.read_csv(ratings_url,sep='\t', low_memory=False)
akas = pd.read_csv(akas_url,sep='\t', low_memory=False)

# Basics

In [4]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9057835 entries, 0 to 9057834
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 622.0+ MB


In [5]:
basics['titleType'].unique()

array(['short', 'movie', 'tvEpisode', 'tvSeries', 'tvShort', 'tvMovie',
       'tvMiniSeries', 'tvSpecial', 'video', 'videoGame', 'tvPilot'],
      dtype=object)

In [6]:
basics.head()

tconst titleType            primaryTitle           originalTitle  \
0  tt0000001     short              Carmencita              Carmencita   
1  tt0000002     short  Le clown et ses chiens  Le clown et ses chiens   
2  tt0000003     short          Pauvre Pierrot          Pauvre Pierrot   
3  tt0000004     short             Un bon bock             Un bon bock   
4  tt0000005     short        Blacksmith Scene        Blacksmith Scene   

  isAdult startYear endYear runtimeMinutes                    genres  
0       0      1894      \N              1         Documentary,Short  
1       0      1892      \N              5           Animation,Short  
2       0      1892      \N              4  Animation,Comedy,Romance  
3       0      1892      \N             12           Animation,Short  
4       0      1893      \N              1              Comedy,Short

In [7]:
basics.replace({'\\\\N':np.nan},inplace=True)

In [8]:
basics.replace({'\\N':np.nan},inplace=True)

In [9]:
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle           11
originalTitle          11
isAdult                 1
startYear         1202182
endYear           8964097
runtimeMinutes    6618305
genres             412366
dtype: int64

In [10]:
basics.dropna(subset=['runtimeMinutes','genres', 'startYear'],inplace=True)

In [11]:
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle            0
originalTitle           0
isAdult                 0
startYear               0
endYear           2292178
runtimeMinutes          0
genres                  0
dtype: int64

In [12]:
basics = basics[basics['titleType'] == 'movie']

In [13]:
basics['titleType'].unique()

array(['movie'], dtype=object)

In [14]:
basics['startYear'] = basics['startYear'].astype(str).astype(int)

In [15]:
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]

In [16]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 272931 entries, 570 to 9057684
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          272931 non-null  object
 1   titleType       272931 non-null  object
 2   primaryTitle    272931 non-null  object
 3   originalTitle   272931 non-null  object
 4   isAdult         272931 non-null  object
 5   startYear       272931 non-null  int32 
 6   endYear         0 non-null       object
 7   runtimeMinutes  272931 non-null  object
 8   genres          272931 non-null  object
dtypes: int32(1), object(8)
memory usage: 19.8+ MB


In [17]:
year_upper_filter = basics['startYear'] <= 2022
year_lower_filter = basics['startYear'] >= 2000

In [18]:
basics = basics.loc[year_upper_filter & year_lower_filter, :]
basics  

tconst titleType  \
34805    tt0035423     movie   
61119    tt0062336     movie   
67672    tt0069049     movie   
77968    tt0079644     movie   
86806    tt0088751     movie   
...            ...       ...   
9057507  tt9916170     movie   
9057516  tt9916190     movie   
9057555  tt9916270     movie   
9057600  tt9916362     movie   
9057684  tt9916538     movie   

                                              primaryTitle  \
34805                                       Kate & Leopold   
61119    The Tango of the Widower and Its Distorting Mi...   
67672                           The Other Side of the Wind   
77968                                        November 1828   
86806                                    The Naked Monster   
...                                                    ...   
9057507                                      The Rehearsal   
9057516                                          Safeguard   
9057555                           Il talento del calabrone   
9057600                                              Coven   
9057684                                Kuambil Lagi Hatiku   

                                     originalTitle isAdult  startYear endYear  \
34805                               Kate & Leopold       0       2001     NaN   
61119    El Tango del Viudo y Su Espejo Deformante       0       2020     NaN   
67672                   The Other Side of the Wind       0       2018     NaN   
77968                                November 1828       0       2001     NaN   
86806                            The Naked Monster       0       2005     NaN   
...                                            ...     ...        ...     ...   
9057507                                   O Ensaio       0       2019     NaN   
9057516                                  Safeguard       0       2020     NaN   
9057555                   Il talento del calabrone       0       2020     NaN   
9057600                                   Akelarre       0       2020     NaN   
9057684                        Kuambil Lagi Hatiku       0       2019     NaN   

        runtimeMinutes                     genres  
34805              118     Comedy,Fantasy,Romance  
61119               70                      Drama  
67672              122                      Drama  
77968              140                  Drama,War  
86806              100       Comedy,Horror,Sci-Fi  
...                ...                        ...  
9057507             51                      Drama  
9057516             90  Action,Adventure,Thriller  
9057555             84                   Thriller  
9057600             92              Drama,History  
9057684            123                      Drama  

[140778 rows x 9 columns]

In [19]:
basics.drop(columns=['endYear'], axis = 1,inplace=True)

In [20]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140778 entries, 34805 to 9057684
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          140778 non-null  object
 1   titleType       140778 non-null  object
 2   primaryTitle    140778 non-null  object
 3   originalTitle   140778 non-null  object
 4   isAdult         140778 non-null  object
 5   startYear       140778 non-null  int32 
 6   runtimeMinutes  140778 non-null  object
 7   genres          140778 non-null  object
dtypes: int32(1), object(7)
memory usage: 9.1+ MB


# AKAs


In [22]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32571504 entries, 0 to 32571503
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   titleId          object
 1   ordering         int64 
 2   title            object
 3   region           object
 4   language         object
 5   types            object
 6   attributes       object
 7   isOriginalTitle  object
dtypes: int64(1), object(7)
memory usage: 1.9+ GB


In [23]:
akas.replace({'\\N':np.nan},inplace=True) 
akas.head()

titleId  ordering                      title region language  \
0  tt0000001         1                 Карменсіта     UA      NaN   
1  tt0000001         2                 Carmencita     DE      NaN   
2  tt0000001         3  Carmencita - spanyol tánc     HU      NaN   
3  tt0000001         4                 Καρμενσίτα     GR      NaN   
4  tt0000001         5                 Карменсита     RU      NaN   

         types     attributes isOriginalTitle  
0  imdbDisplay            NaN               0  
1          NaN  literal title               0  
2  imdbDisplay            NaN               0  
3  imdbDisplay            NaN               0  
4  imdbDisplay            NaN               0

In [24]:
akas.isna().sum()

titleId                   0
ordering                  0
title                     4
region              1930921
language            6250018
types              27211131
attributes         32329126
isOriginalTitle        2187
dtype: int64

In [ ]:
#akas.dropna(subset=['title','region', 'language', 'types', 'attributes','isOriginalTitle'],inplace=True)

In [ ]:
#akas.isna().sum()

In [25]:
akas = akas[akas['region'] == 'US']
akas.head()

titleId  ordering                   title region language        types  \
5   tt0000001         6              Carmencita     US      NaN  imdbDisplay   
14  tt0000002         7  The Clown and His Dogs     US      NaN          NaN   
33  tt0000005        10        Blacksmith Scene     US      NaN  imdbDisplay   
36  tt0000005         1     Blacksmithing Scene     US      NaN  alternative   
41  tt0000005         6     Blacksmith Scene #1     US      NaN  alternative   

               attributes isOriginalTitle  
5                     NaN               0  
14  literal English title               0  
33                    NaN               0  
36                    NaN               0  
41                    NaN               0

In [26]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1334104 entries, 5 to 32571248
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1334104 non-null  object
 1   ordering         1334104 non-null  int64 
 2   title            1334104 non-null  object
 3   region           1334104 non-null  object
 4   language         3611 non-null     object
 5   types            1026581 non-null  object
 6   attributes       44416 non-null    object
 7   isOriginalTitle  1332729 non-null  object
dtypes: int64(1), object(7)
memory usage: 91.6+ MB


# Ratings

In [27]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1253649 entries, 0 to 1253648
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1253649 non-null  object 
 1   averageRating  1253649 non-null  float64
 2   numVotes       1253649 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 28.7+ MB


In [28]:
ratings.head()

tconst  averageRating  numVotes
0  tt0000001            5.7      1892
1  tt0000002            5.9       253
2  tt0000003            6.5      1686
3  tt0000004            5.7       166
4  tt0000005            6.2      2503

In [30]:
ratings.replace({'\\N':np.nan},inplace=True) 
ratings.head()

tconst  averageRating  numVotes
0  tt0000001            5.7      1892
1  tt0000002            5.9       253
2  tt0000003            6.5      1686
3  tt0000004            5.7       166
4  tt0000005            6.2      2503

In [31]:
ratings.isna().sum()

tconst           0
averageRating    0
numVotes         0
dtype: int64

# Filtering

In [32]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers = basics['tconst'].isin(akas['titleId'])
keepers

34805       True
61119       True
67672       True
77968      False
86806       True
           ...  
9057507     True
9057516     True
9057555    False
9057600     True
9057684    False
Name: tconst, Length: 140778, dtype: bool

In [34]:
basics = basics[keepers]
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81687 entries, 34805 to 9057600
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tconst          81687 non-null  object
 1   titleType       81687 non-null  object
 2   primaryTitle    81687 non-null  object
 3   originalTitle   81687 non-null  object
 4   isAdult         81687 non-null  object
 5   startYear       81687 non-null  int32 
 6   runtimeMinutes  81687 non-null  object
 7   genres          81687 non-null  object
dtypes: int32(1), object(7)
memory usage: 5.3+ MB


# Saving file

In [35]:
# example making new folder with os
import os
os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")

['.ipynb_checkpoints']

In [36]:
## Save current dataframe to file.
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

In [37]:
# Open saved file and preview again
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
basics.head()

tconst titleType                                       primaryTitle  \
0  tt0035423     movie                                     Kate & Leopold   
1  tt0062336     movie  The Tango of the Widower and Its Distorting Mi...   
2  tt0069049     movie                         The Other Side of the Wind   
3  tt0088751     movie                                  The Naked Monster   
4  tt0093119     movie                                Grizzly II: Revenge   

                               originalTitle  isAdult  startYear  \
0                             Kate & Leopold        0       2001   
1  El Tango del Viudo y Su Espejo Deformante        0       2020   
2                 The Other Side of the Wind        0       2018   
3                          The Naked Monster        0       2005   
4                   Grizzly II: The Predator        0       2020   

   runtimeMinutes                  genres  
0             118  Comedy,Fantasy,Romance  
1              70                   Drama  
2             122                   Drama  
3             100    Comedy,Horror,Sci-Fi  
4              74   Horror,Music,Thriller

In [38]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81687 entries, 0 to 81686
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tconst          81687 non-null  object
 1   titleType       81687 non-null  object
 2   primaryTitle    81687 non-null  object
 3   originalTitle   81687 non-null  object
 4   isAdult         81687 non-null  int64 
 5   startYear       81687 non-null  int64 
 6   runtimeMinutes  81687 non-null  int64 
 7   genres          81687 non-null  object
dtypes: int64(3), object(5)
memory usage: 5.0+ MB


In [39]:
## Save current dataframe to file.
ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)

In [40]:
# Open saved file and preview again
ratings = pd.read_csv("Data/title_ratings.csv.gz", low_memory = False)
ratings.head()

tconst  averageRating  numVotes
0  tt0000001            5.7      1892
1  tt0000002            5.9       253
2  tt0000003            6.5      1686
3  tt0000004            5.7       166
4  tt0000005            6.2      2503

In [41]:
## Save current dataframe to file.
akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)

In [42]:
# Open saved file and preview again
akas = pd.read_csv("Data/title_akas.csv.gz", low_memory = False)
akas.head()

titleId  ordering                   title region language        types  \
0  tt0000001         6              Carmencita     US      NaN  imdbDisplay   
1  tt0000002         7  The Clown and His Dogs     US      NaN          NaN   
2  tt0000005        10        Blacksmith Scene     US      NaN  imdbDisplay   
3  tt0000005         1     Blacksmithing Scene     US      NaN  alternative   
4  tt0000005         6     Blacksmith Scene #1     US      NaN  alternative   

              attributes  isOriginalTitle  
0                    NaN              0.0  
1  literal English title              0.0  
2                    NaN              0.0  
3                    NaN              0.0  
4                    NaN              0.0

In [43]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1334104 entries, 0 to 1334103
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   titleId          1334104 non-null  object 
 1   ordering         1334104 non-null  int64  
 2   title            1334104 non-null  object 
 3   region           1334104 non-null  object 
 4   language         3611 non-null     object 
 5   types            1026581 non-null  object 
 6   attributes       44416 non-null    object 
 7   isOriginalTitle  1332729 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 81.4+ MB


# API


In [ ]:
import json
with open('/Users/robo1/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

In [ ]:
import tmdbsimple as tmdb
tmdb.API_KEY =  login['api-key']

In [ ]:
movie = tmdb.Movies('tt1361336')
info = movie.info()
info['budget']

In [ ]:
# example from package README
response = movie.releases()
for c in movie.countries:
    if c['iso_3166_1'] == 'US':
        print(c['certification'])

In [ ]:
# Get the movie object for the current id
movie = tmdb.Movies('tt1361336')
# save the .info .releases dictionaries
info = movie.info()
releases = movie.releases()
# Loop through countries in releases
for c in releases['countries']:
    # if the country abbreviation==US
    if c['iso_3166_1' ] =='US':
        ## save a "certification" key in the info dict with the certification
       info['certification'] = c['certification']

In [ ]:
def get_movie_with_rating(movie_id):
    #gets movie id and release dates
    movie = tmdb.Movies(movie_id)
    #create output dictionary
    movie_info = movie.info()
    releases = movie.releases()
    for c in releases['countries']:
        if c['iso_3166_1']=='US':
            movie_info['certification'] = c['certification']
            return movie_info

In [ ]:
test = get_movie_with_rating("tt0848228") #put your function name here
test

# LOOPS


In [45]:
import json
with open('/Users/robo1/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['api-key'])

In [46]:
import tmdbsimple as tmdb
tmdb.API_KEY =  login['api-key']

In [74]:
def write_json(new_data, filename):
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
            file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [77]:
def get_movie_with_rating(movie_id):
    #gets movie id and release dates
    movie = tmdb.Movies(movie_id)
    #create output dictionary
    movie_info = movie.info()
    releases = movie.releases()
    for c in releases['countries']:
        if c['iso_3166_1' ] =='US':
            movie_info['certification'] = c['certification']
    return movie_info

In [80]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    #num = 0
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)
    # If it does not exist: create it
    if file_exists == False:
        # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
    # Load in the dataframe from project part 1 as basics:
        basics = pd.read_csv('Data/title_basics.csv.gz')
    # Saving new year as the current df
        df = basics.loc[basics['startYear']==YEAR].copy()
    # saving movie ids to list
        movie_ids = df['tconst'].copy()
    # Load existing data from json into a dataframe called "previous_df"
        previous_df = pd.read_json(JSON_FILE)
    # filter out any ids that are already in the JSON_FILE
        movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    #######################################################################################################################
    #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,desc=f'Movies from {YEAR}',position=1,leave=True):
        #num += 1
        #if num > 20:
            #break
        # Attempt to retrieve then data for the movie id
        try:
            temp = get_movie_with_rating(movie_id)  #This uses your pre-made function
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        # If it fails,  make a dict with just the id and None for certification.
        except Exception as e:
            continue
    ##############################################################################################################           
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1525 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/1525 [00:00<?, ?it/s]